In [2]:
import pandas as pd
import requests
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import shapely
from shapely import geometry
import re
import time

In [3]:
from fake_useragent import UserAgent

In [4]:
#логер
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [5]:
df = pd.read_csv('../data/places_GP.csv')

In [4]:
gdf = gpd.GeoDataFrame(df)

In [5]:
gdf.geometry = [shapely.wkt.loads(r['geometry']) for i, r in gdf.iterrows()]

In [148]:
no_photos

[]

In [147]:
df.loc[df.count_photo.isna()]

,url,title,address,category,popular_times_bars,geometry,types,rating,user_ratings_total,count_photo
0,https://www.google.com/maps/place/?q=place_id:...,NaN,Об этих данных,NaN,NaN,POINT (39.06791 45.01079),"['laundry', 'point_of_interest', 'establishment']",2.4,8.0,NaN
1,https://www.google.com/maps/place/?q=place_id:...,"Новый век, СФ","ул. Демуса, 60, Краснодар, Краснодарский край,...",Строительная компания,NaN,POINT (39.09853 45.04941),"['general_contractor', 'point_of_interest', 'e...",5.0,2.0,NaN
2,https://www.google.com/maps/place/?q=place_id:...,Магазин автоприцепов,"ул. Ростовское ш., 24/1, Краснодар, Краснодарс...",Продажа прицепов,NaN,POINT (38.99111 45.08963),"['point_of_interest', 'store', 'establishment']",4.5,2.0,NaN
3,https://www.google.com/maps/place/?q=place_id:...,Pepe магазин детской одежды,"Краснодар, Краснодарский край, 350901",Магазин одежды,NaN,POINT (39.03829 45.07438),"['point_of_interest', 'establishment']",NaN,NaN,NaN
4,https://www.google.com/maps/place/?q=place_id:...,Трейд Инвест,"ул. Тихорецкая, 8, Краснодар, Краснодарский кр...",Оптовая продажа строительных материалов,NaN,POINT (39.03771 45.03959),"['general_contractor', 'point_of_interest', 'e...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
15515,https://www.google.com/maps/place/?q=place_id:...,"Полезная Площадь, агентство недвижимости","пр. Чекистов, 38, Краснодар, Краснодарский кра...",Агентство недвижимости,NaN,POINT (38.91878 45.03238),"['real_estate_agency', 'point_of_interest', 'e...",0.0,0.0,NaN
15516,https://www.google.com/maps/place/?q=place_id:...,Barcelo Biagi,"ТРК ""Красная площадь"", ул. Дзержинского, 100, ...",Обувной магазин,NaN,POINT (38.98488 45.10173),"['shoe_store', 'point_of_interest', 'store', '...",0.0,0.0,NaN
15517,https://www.google.com/maps/place/?q=place_id:...,"Амк, Ооо","Рождественская наб., д.2, Краснодар, Краснодар...",NaN,NaN,POINT (38.90718 45.02488),"['point_of_interest', 'establishment']",0.0,0.0,NaN
15519,https://www.google.com/maps/place/?q=place_id:...,Сеть ателье Дамское счастье,"ул. Кубанская набережная, 19, Краснодар, Красн...",Портной,NaN,POINT (38.95940 45.02063),"['point_of_interest', 'clothing_store', 'store...",0.0,0.0,NaN


In [19]:
d = {0:[],1:[],2:[],3:[],4:[],5:[],6:[]}
for i, r in df.loc[~df.count_photo.astype(str).str.isnumeric()& ~df.count_photo.isna()].sort_values('user_ratings_total', ascending = False, ignore_index=True).iterrows():
    d[i%7].append(r['url'])

In [20]:
def get_url(url):
    session = requests.Session()
    session.headers.update({
        'Referer':'https://www.google.com/',
        'User-Agent': UserAgent().chrome,
        'Cookie': 'HSID=A8eREZIFT1JEvs0QY; SSID=AkT63pRefd9zZ9EyM; APISID=vRnLDNULmcW8mJwb/A4AlXkbmjGCmFE1hP; SAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; __Secure-1PAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; __Secure-3PAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; SID=CwhAuCnOmS_V-kzHjIiF_l9DcLUZiXYMtsHwCnYNp0yz1HR5bFib6uPJJ8ikC9worT2NQA.; __Secure-1PSID=CwhAuCnOmS_V-kzHjIiF_l9DcLUZiXYMtsHwCnYNp0yz1HR56PNuz1ULgmXYdFHDEuOwBg.; __Secure-3PSID=CwhAuCnOmS_V-kzHjIiF_l9DcLUZiXYMtsHwCnYNp0yz1HR5RVGobiema13Yr7y1ovYZRA.; SEARCH_SAMESITE=CgQI25MB; NID=511=nRbY_-aARIEJ8BC2lsWaY5lZiAbsCfwYmi8Y5U3ysQnn35eW-wANOCdGo2Yqxwq7CYlHYal39i3BOuYBowX3TUs65FQXDtAxd3AaAow3hiTNDCSK8GkpoTqmgRUTy4ESBONWNSMlG-2vGbHNkVUnLgeskPFhfXOJadr8-4_M78lTK_YtvdfLeBJLEoe4y9X31CLIUDdwc1Afob3Ma2z3kzH40vvRIY95iLw_Hk1qmmmQYrLe6awCyYsNLkOrwigoDoTTYCvWPDc_A5nCdgRuFs9p; 1P_JAR=2021-10-12-04; SIDCC=AJi4QfGhPr0AEv0NSDMl9WdY6D70o5TLwiZlmE5vYCDO0HqTMUDpk6D9-J0rS-EzqLQnDXCrHXeD; __Secure-3PSIDCC=AJi4QfHmIUFMX1j8uZvqqKxxPDbO_wq-TwWifjplEMZNayU5_BZ3FM_TrpwMVEG1FL6thixrwe8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ru,en;q=0.9',
#            'x-csrftoken':''
    })
    return session.get(url)

In [21]:
no_photos = []

In [22]:
def get_count_photo(urls):
    for url in log_progress(urls, 1):
        time.sleep(3)
        try:
            text = get_url(url).text
            count_photos.append({'url':url, 'count_photo':int(re.findall('\d+\sфото',text)[0][:-5])})
        except:
            no_photos.append(url)

In [23]:
# datalist = gdf.url
count_photos = []

if __name__== "__main__":
    
    freeze_support()
    pool = Pool()

    # max number of parallel process
#     ITERATION_COUNT = cpu_count()-1 if len(datalist) >= cpu_count()-1 else len(datalist)

#     count_per_iteration = len(datalist) / float(ITERATION_COUNT)
    ITERATION_COUNT = 7
    for i in range(0, ITERATION_COUNT):
#         list_start = int(count_per_iteration * i)
#         list_end = int(count_per_iteration * (i+1))
        pool.apply_async(get_count_photo, [d[i]])

In [24]:
ph_df = pd.DataFrame(count_photos)
ph_df

,url,count_photo
0,https://www.google.com/maps/place/?q=place_id:...,4
1,https://www.google.com/maps/place/?q=place_id:...,3
2,https://www.google.com/maps/place/?q=place_id:...,4


In [25]:
df.loc[df.url.isin(ph_df.url), 'count_photo'] = df.loc[df.url.isin(ph_df.url)].url.apply(lambda x: ph_df.loc[ph_df.url==x, 'count_photo'].to_list()[0])

In [57]:
df = df.merge(pd.DataFrame(count_photos), how = 'left')

In [26]:
df.loc[~df.count_photo.astype(str).str.isnumeric()& ~df.count_photo.isna()]

,url,title,address,category,popular_times_bars,geometry,types,rating,user_ratings_total,count_photo,new_category,new_category_2


In [27]:
df.to_csv('../data/places_GP.csv', index = False)

# Проверка

In [196]:
response = get_url(d[1][1])
for key, value in response.request.headers.items():
    print(key+": "+value)

User-Agent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.14 (KHTML, like Gecko) Chrome/24.0.1292.0 Safari/537.14
accept-encoding: gzip, deflate, br
Accept: */*
Connection: keep-alive
Referer: https://www.google.com/
Cookie: OGPC=19022552-1:; SEARCH_SAMESITE=CgQIsJMB; OTZ=6173585_24_24_123480_20_435960; SID=CwhAuFzdhWtATdALcwrJ92VWCLgNSOqLoISsQXkA3VvPZIiUtXM7lwEUgH_wIM2jvW_hbQ.; __Secure-1PSID=CwhAuFzdhWtATdALcwrJ92VWCLgNSOqLoISsQXkA3VvPZIiUHYM-x3RnYAMpE71JooiCOQ.; __Secure-3PSID=CwhAuFzdhWtATdALcwrJ92VWCLgNSOqLoISsQXkA3VvPZIiUGesm0V1ctZmg6qxDxALDmw.; HSID=Ac9lsgaaIPmHh_RhQ; SSID=AT1ZDeLhsjTHJLUml; APISID=vRnLDNULmcW8mJwb/A4AlXkbmjGCmFE1hP; SAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; __Secure-1PAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; __Secure-3PAPISID=U3SWIc7cG1mczSbk/AcIXcRKywUm5o06MY; NID=511=WPGa7CgDlx3ETgzyJ-YR786FOV6gqUyHVWmDUlHykM_obpattvk6xcpUnkI2Gb86I-9iDnGRd0bnh0ZH1lTf78fyAcclhpiHfcHARMzDgajPxBO2XqdGYvAS3l5QMMtIydaGJAp2TE6X6Ugo4LhRllmtPZLTTy-QHx1_AhVVW_mvN7v4s9n0lVH--

In [197]:
response.text

'<!DOCTYPE html><html itemscope="" itemtype="http://schema.org/Place" lang="ru"> <head> <link href="/maps/preview/opensearch.xml?hl=ru" title="Google Карты" rel="search" type="application/opensearchdescription+xml"> <title> Google Карты </title> <meta content="Найти информацию о местных компаниях, посмотреть карты и получить указания о маршруте в Картах Google." name="Description">  <meta content="initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"> <meta content="chrome=1" http-equiv="X-UA-Compatible"> <meta content="notranslate" name="google"> <meta content="origin" name="referrer"> <meta content="ByHT0GXztW_RcGxS0o86DBf1WtNu02FfqlcT8njnSqU" name="google-site-verification"> <meta content="Diln__r3p9-tt39P2Cl2Amvx6oFB4PATnxuFBaw6ej8" name="google-site-verification">   <meta content="Магнит · ул. Российская, 79/1, Краснодар, Краснодарский край, 350029" itemprop="name"> <meta content="Магнит · ул. Российская, 79/1, Краснодар, Краснодарский край, 35